In [1]:
import pathlib
from utils import get_filenames_of_path

In [2]:
# imports

import numpy as np
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn.datasets import ObjectDetectionDataSet
from pytorch_faster_rcnn.transformations import Clip, ComposeDouble
from pytorch_faster_rcnn.transformations import FunctionWrapperDouble
from pytorch_faster_rcnn.transformations import normalize_01
from pytorch_faster_rcnn.utils import get_filenames_of_path
from pytorch_faster_rcnn.utils import stats_dataset
from pytorch_faster_rcnn.visual import DatasetViewer

In [3]:
# root directory
root = pathlib.Path('/home/bdwumah74/Indiv_Proj/Dataset')

In [4]:
# input and target files
inputs = get_filenames_of_path(root / 'input')
targets = get_filenames_of_path(root / 'target')

inputs.sort()
targets.sort()


In [5]:
# mapping
mapping = {
    'HCC': 1
    }

In [6]:
# transforms
transforms = ComposeDouble([
    Clip(),
    # AlbumentationWrapper(albumentation=A.HorizontalFlip(p=0.5)),
    # AlbumentationWrapper(albumentation=A.RandomScale(p=0.5, scale_limit=0.5)),
    # AlbumentationWrapper(albumentation=A.VerticalFlip(p=0.5)),
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [7]:
# dataset building
dataset = ObjectDetectionDataSet(inputs=inputs,
                                 targets=targets,
                                 transform=transforms,
                                 use_cache=False,
                                 convert_to_format=None,
                                 mapping=mapping)

In [8]:
transform = GeneralizedRCNNTransform(min_size=1024,
                                     max_size=1024,
                                     image_mean=[0.485, 0.456, 0.406],
                                     image_std=[0.229, 0.224, 0.225])

stats_transform = stats_dataset(dataset, transform)
stats_transform

{'image_height': tensor([768., 768., 768., 768., 768., 768., 768., 768., 768., 768., 768., 768.,
         768., 768., 768., 768., 768., 576., 576., 768., 768., 768., 768., 768.,
         768., 768., 768., 768., 768., 768., 768., 768., 768., 768., 768., 768.,
         768., 768., 768., 768., 768., 768., 768., 768., 768., 768., 768., 768.,
         768., 768., 768., 768., 768., 768., 768., 768., 768., 768., 768.]),
 'image_width': tensor([1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024.,
         1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024.,
         1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024.,
         1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024.,
         1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024.,
         1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024.]),
 'image_mean': tensor([-1.5360, -1.5222, -1.5755, -1.2957, -1.4222, -1.4464, -1.4456, -1.4176,
    

In [9]:
from itertools import chain

import pytorch_lightning as pl
import torch

from utils import from_dict_to_boundingbox

In [10]:
class FasterRCNN_lightning(pl.LightningModule):
    def __init__(self,
                 model: torch.nn.Module,
                 lr: float = 0.0001,
                 iou_threshold: float = 0.5
                 ):
        super().__init__()

        # Model
        self.model = model

        # Classes (background inclusive)
        self.num_classes = self.model.num_classes

        # Learning rate
        self.lr = lr

        # IoU threshold
        self.iou_threshold = iou_threshold

        # Transformation parameters
        self.mean = model.image_mean
        self.std = model.image_std
        self.min_size = model.min_size
        self.max_size = model.max_size

        # Save hyperparameters
        self.save_hyperparameters()

    def forward(self, x):
        self.model.eval()
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # Batch
        x, y, x_name, y_name = batch  # tuple unpacking

        loss_dict = self.model(x, y)
        loss = sum(loss for loss in loss_dict.values())

        self.log_dict(loss_dict)
        return loss

    def validation_step(self, batch, batch_idx):
        # Batch
        x, y, x_name, y_name = batch

        # Inference
        preds = self.model(x)

        gt_boxes = [from_dict_to_BoundingBox(target, name=name, groundtruth=True) for target, name in zip(y, x_name)]
        gt_boxes = list(chain(*gt_boxes))

        pred_boxes = [from_dict_to_BoundingBox(pred, name=name, groundtruth=False) for pred, name in zip(preds, x_name)]
        pred_boxes = list(chain(*pred_boxes))

        return {'pred_boxes': pred_boxes, 'gt_boxes': gt_boxes}

    def validation_epoch_end(self, outs):
        gt_boxes = [out['gt_boxes'] for out in outs]
        gt_boxes = list(chain(*gt_boxes))
        pred_boxes = [out['pred_boxes'] for out in outs]
        pred_boxes = list(chain(*pred_boxes))

        from metrics.pascal_voc_evaluator import get_pascalvoc_metrics
        from metrics.enumerators import MethodAveragePrecision
        metric = get_pascalvoc_metrics(gt_boxes=gt_boxes,
                                       det_boxes=pred_boxes,
                                       iou_threshold=self.iou_threshold,
                                       method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION,
                                       generate_table=True)

        per_class, mAP = metric['per_class'], metric['mAP']
        self.log('Validation_mAP', mAP)

        for key, value in per_class.items():
            self.log(f'Validation_AP_{key}', value['AP'])

    def test_step(self, batch, batch_idx):
        # Batch
        x, y, x_name, y_name = batch

        # Inference
        preds = self.model(x)

        gt_boxes = [from_dict_to_BoundingBox(target, name=name, groundtruth=True) for target, name in zip(y, x_name)]
        gt_boxes = list(chain(*gt_boxes))

        pred_boxes = [from_dict_to_BoundingBox(pred, name=name, groundtruth=False) for pred, name in zip(preds, x_name)]
        pred_boxes = list(chain(*pred_boxes))

        return {'pred_boxes': pred_boxes, 'gt_boxes': gt_boxes}

    def test_epoch_end(self, outs):
        gt_boxes = [out['gt_boxes'] for out in outs]
        gt_boxes = list(chain(*gt_boxes))
        pred_boxes = [out['pred_boxes'] for out in outs]
        pred_boxes = list(chain(*pred_boxes))

        from metrics.pascal_voc_evaluator import get_pascalvoc_metrics
        from metrics.enumerators import MethodAveragePrecision
        metric = get_pascalvoc_metrics(gt_boxes=gt_boxes,
                                       det_boxes=pred_boxes,
                                       iou_threshold=self.iou_threshold,
                                       method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION,
                                       generate_table=True)

        per_class, mAP = metric['per_class'], metric['mAP']
        self.log('Test_mAP', mAP)

        for key, value in per_class.items():
            self.log(f'Test_AP_{key}', value['AP'])

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(),
                                    lr=self.lr,
                                    momentum=0.9,
                                    weight_decay=0.005)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                                  mode='max',
                                                                  factor=0.75,
                                                                  patience=30,
                                                                  min_lr=0)
        return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler, 'monitor': 'Validation_mAP'}

In [11]:
# imports
import os
import pathlib
from dataclasses import dataclass, field
from typing import List, Optional, Tuple

import albumentations as albu
import numpy as np
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)

In [12]:
from pytorch_lightning.loggers.neptune import NeptuneLogger
from torch.utils.data import DataLoader

from pytorch_faster_rcnn.backbone_resnet import ResNetBackbones
from pytorch_faster_rcnn.backbone_mobilenet import MobileNetBackbones
from pytorch_faster_rcnn.datasets import ObjectDetectionDataSet
from pytorch_faster_rcnn.faster_RCNN import (
    FasterRCNNLightning,
    get_faster_rcnn_resnet,
    get_faster_rcnn_mobilenet
)

In [13]:
from pytorch_faster_rcnn.transformations import (
    AlbumentationWrapper,
    Clip,
    ComposeDouble,
    FunctionWrapperDouble,
    normalize_01,
)

In [14]:
from pytorch_faster_rcnn.utils import (
    collate_double,
    get_filenames_of_path,
    log_mapping_neptune,
    log_model_neptune,
    log_packages_neptune,
)

In [15]:
# hyper-parameters
@dataclass
class Params:
    BATCH_SIZE: int = 2
    OWNER: str = "bdwumah"  # set your name here, e.g. johndoe22
    SAVE_DIR: Optional[
        str
    ] = None  # checkpoints will be saved to cwd (current working directory)
    LOG_MODEL: bool = False  # whether to log the model to neptune after training
    GPU: Optional[int] = None  # set to None for cpu training
    LR: float = 0.001
    PRECISION: int = 32
    CLASSES: int = 2
    SEED: int = 42
    PROJECT: str = "bdwumah/Individual-Project-1"
    EXPERIMENT: str = "MobileNet"
    MAXEPOCHS: int = 150
    PATIENCE: int = 50
    BACKBONE: MobileNetBackbones = MobileNetBackbones.MOBILENETV2
    FPN: bool = False
    ANCHOR_SIZE: Tuple[Tuple[int, ...], ...] = ((32, 64, 128, 256, 512),)
    ASPECT_RATIOS: Tuple[Tuple[float, ...]] = ((0.5, 1.0, 2.0),)
    MIN_SIZE: int = 1024
    MAX_SIZE: int = 1024
    IMG_MEAN: List = field(default_factory=lambda: [0.485, 0.456, 0.406])
    IMG_STD: List = field(default_factory=lambda: [0.229, 0.224, 0.225])
    IOU_THRESHOLD: float = 0.5

In [16]:
# root directory
ROOT_PATH = pathlib.Path.cwd()

In [17]:
params = Params()

In [18]:
# save directory
save_dir = os.getcwd() if not params.SAVE_DIR else params.SAVE_DIR

In [19]:
# root directory
root = ROOT_PATH / "Dataset"

In [20]:
# input and target files
inputs = get_filenames_of_path(root / 'input')
targets = get_filenames_of_path(root / 'target')

inputs.sort()
targets.sort()

In [21]:
# training transformations and augmentations
transforms_training = ComposeDouble(
    [
        Clip(),
        AlbumentationWrapper(albumentation=albu.HorizontalFlip(p=0.5)),
        AlbumentationWrapper(
            albumentation=albu.RandomScale(p=0.5, scale_limit=0.5)
        ),
        # AlbuWrapper(albu=A.VerticalFlip(p=0.5)),
        FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(normalize_01),
    ]
)

In [22]:
# validation transformations
transforms_validation = ComposeDouble([
    Clip(),
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [23]:
# test transformations
transforms_test = ComposeDouble([
    Clip(),
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [24]:
# random seed
seed_everything(params.SEED)

Global seed set to 42


42

In [25]:
# training validation test split
inputs_train, inputs_valid, inputs_test = inputs[:25], inputs[25:50], inputs[50:]
targets_train, targets_valid, targets_test = targets[:25], targets[25:50], targets[50:]

In [26]:
# dataset training
dataset_train = ObjectDetectionDataSet(inputs=inputs_train,
                                       targets=targets_train,
                                       transform=transforms_training,
                                       use_cache=True,
                                       convert_to_format=None,
                                       mapping=mapping)

In [27]:
# dataset validation
dataset_valid = ObjectDetectionDataSet(inputs=inputs_valid,
                                       targets=targets_valid,
                                       transform=transforms_validation,
                                       use_cache=True,
                                       convert_to_format=None,
                                       mapping=mapping)

In [28]:
# dataset test
dataset_test = ObjectDetectionDataSet(inputs=inputs_test,
                                      targets=targets_test,
                                      transform=transforms_test,
                                      use_cache=True,
                                      convert_to_format=None,
                                      mapping=mapping)

In [29]:
# dataloader training
dataloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=params.BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_double,
)

In [30]:
# dataloader validation
dataloader_valid = DataLoader(dataset=dataset_valid,
                              batch_size=1,
                              shuffle=False,
                              num_workers=0,
                              collate_fn=collate_double)

In [31]:
# dataloader test
dataloader_test = DataLoader(dataset=dataset_test,
                             batch_size=1,
                             shuffle=False,
                             num_workers=0,
                             collate_fn=collate_double)

In [32]:
api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4MjEwNWM2NC1jNmM5LTQyYTctOGRlNy1hN2M0YTk2MmY0N2QifQ==",

In [33]:
#from neptunecontrib.monitoring.pytorch_lightning import NeptuneLogger

neptune_logger = NeptuneLogger(
    project="bdwumah/Individual-Project-1",
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4MjEwNWM2NC1jNmM5LTQyYTctOGRlNy1hN2M0YTk2MmY0N2QifQ==",
    log_model_checkpoints=True,
)

neptune_logger.log_hyperparams(params=params.__dict__)

assert neptune_logger.name  # http GET request to check if the project exists

https://app.neptune.ai/bdwumah/Individual-Project-1/e/IN-67


Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/you-should-know/what-can-you-log-and-display#hardware-consumption


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [34]:
# model init for mobilenet
model = get_faster_rcnn_mobilenet(
    num_classes=params.CLASSES,
    backbone_name=params.BACKBONE,
    anchor_size=params.ANCHOR_SIZE,
    aspect_ratios=params.ASPECT_RATIOS,
    fpn=params.FPN,
    min_size=params.MIN_SIZE,
    max_size=params.MAX_SIZE,
)

In [35]:
'''# model init for resnet
model = get_faster_rcnn_resnet(
    num_classes=params.CLASSES,
    backbone_name=params.BACKBONE,
    anchor_size=params.ANCHOR_SIZE,
    aspect_ratios=params.ASPECT_RATIOS,
    fpn=params.FPN,
    min_size=params.MIN_SIZE,
    max_size=params.MAX_SIZE,
)'''

'# model init for resnet\nmodel = get_faster_rcnn_resnet(\n    num_classes=params.CLASSES,\n    backbone_name=params.BACKBONE,\n    anchor_size=params.ANCHOR_SIZE,\n    aspect_ratios=params.ASPECT_RATIOS,\n    fpn=params.FPN,\n    min_size=params.MIN_SIZE,\n    max_size=params.MAX_SIZE,\n)'

In [36]:
# lightning init
task = FasterRCNNLightning(
    model=model, lr=params.LR, iou_threshold=params.IOU_THRESHOLD
)

In [37]:
# callbacks
checkpoint_callback = ModelCheckpoint(monitor="Validation_mAP", mode="max")
learningrate_callback = LearningRateMonitor(
    logging_interval="step", log_momentum=False
)
early_stopping_callback = EarlyStopping(
    monitor="Validation_mAP", patience=params.PATIENCE, mode="max"
)

In [38]:
# trainer init
trainer = Trainer(
    gpus=params.GPU,
    precision=params.PRECISION,  # try 16 with enable_pl_optimizer=False
    callbacks=[checkpoint_callback, learningrate_callback, early_stopping_callback],
    default_root_dir=save_dir,  # where checkpoints are saved to
    logger=neptune_logger,
    log_every_n_steps=1,
    num_sanity_val_steps=0,
    max_epochs=params.MAXEPOCHS,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [39]:
# start training
trainer.fit(
    model=task, train_dataloaders=dataloader_train, val_dataloaders=dataloader_valid)


  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 1.2 M 
-------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.847     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

In [ ]:
# start testing
trainer.test(ckpt_path="best", dataloaders=dataloader_test)

In [ ]:
# log mapping as table
log_mapping_neptune(mapping=mapping, neptune_logger=neptune_logger)